### 성능 평가

- 평가항목 방식 케이스
    - 지도 학습의 분류
        - 혼동 행렬(정확도, f1-score, 정밀도, 재현율), AUC, ROC
    - 지도 학습의 회귀
        - 손실
    - 비지도 학습
    - 강화 학습

### 머신러닝 > 지도학습 > 분류 > 성능평가지표

- 통계학의 혼동행렬이라는 항목을 이용하여 평가
- 개요
    - 예측값이 실제 관측값을 얼마나 정확하게 예측하였는가?
    - 이 사항을 보여주는 행렬
    - 예
        - 암을 스스로 예측하였는데, 실제 양성인지 악성인지 예측하는 예
        - 실제는 병이 있었는데 없다고 예측, 있다고 예측
        - 실제는 병이 없었는데 있다고 예측, 없다고 예측
|구분|예측:(Positive)|예측:(Negative)|
|:--:|:--:|:--:|
|실제:예(Positive)|TP|FN|
|실제:아니오(Negative)|FP|TN|

- TP : 참긍정
    - 병이 있다고 예측을 했고, 실제로 병이 있었다.
    - 긍정한 것이 참이 되었다.
- TN : 참부정
    - 병이 없다고 예측을 했고, 실제로 병이 없었다.
    - 부정한 것이 참이 되었다.
- FP : 거짓긍정
    - 병이 있다고 예측을 했고, 실제로 병이 없었다.
    - 긍정한 것이 거짓이 되었다.
    - 1형 오류
- FN : 거짓부정
    - 병이 없다고 예측을 했고, 실제로 병이 있었다.
    - 부정한 것이 거짓이 되었다.
    - 2형 오류

- 정밀도(P Precision)
    - 병이 있다고 예측을 했는데, 실제로 병이 있는 비율. 정답율
    - P = TP / (TP + FP)
- 재현율(R Recall)
    - 실제로 병이 있는 전체 데이터 중에 참긍정 비율
    - R = TP / (TP + FN)
- F1 점수
    - 정밀도와 재현율의 조화평균(Harmonic mean)
    - F1 = 2 / (1 / P + 1 / R)
- 특이성
    - 실제 병이 없는 전체 사람들 중에 예측이 TN의 비율
    - TN / (TN + FP)
- 곡선하 면적(ROC) : Area under Curve ROC -> ROC 곡선
    - 참긍정률과 거짓부정률 사이의 관계 표현
    - (1 - 특이성)
    - AUC : ROC의 상세값, 수치

#### 실제 데이터를 이용하여 성능평가를 수행

- 데이터 : iris 활용

In [2]:
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris
import pandas as pd

In [14]:
ds = load_iris()

In [24]:
# 데이터 확인
ds.data.shape, ds.data[:1], ds.feature_names, ds.target_names, ds.target[:3], ds.target[-3:]

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [44]:
# iris 데이터 로드
def attach_iris():
    ds = load_iris()
    df = pd.DataFrame(ds.data, columns=ds.feature_names)
    
    # species 라는 컬럼을 추가한다. -> 값은 setosa, versicolor, virginica 중 하나로 설정
    df['species'] = ds.target_names[ds.target]
#     df['species'] = ds.target
    
    return df

df = attach_iris()
df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


#### 분류 알고리즘을 다양하게 사용하여 교차 성능평가 수행

- 하이퍼 파라미터 튜닝 + 파이프 라인?
- 각각 알고리즘 별로 적용하여 시각화 및 수치 비교? (사용)

In [35]:
import sklearn
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

In [70]:
# 알고리즘
# 향후 알고리즘에 대한 이해와 데이터에 대한 적합성에 대한 식견이 좀 더 보완된다면
# 알고리즘 후보들을 좀 더 적합하게 추릴 수 있을 것이다.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [39]:
# 알고리즘 시각화를 위해 사전 설정
# 선형 그래프의 모양을 지정
# 파라미터 값은 임의 부여
class_map = {
    'LogisticRegression': ('-', LogisticRegression()),
    'DecisionTreeClassifier': ('--', DecisionTreeClassifier(max_depth=5)),
    'RandomForestClassifier': ('.-', RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)),
    'GaussianNB': (':', GaussianNB())
}

In [71]:
for key, value in class_map.items():
    print(key, value)

LogisticRegression ('-', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False))
DecisionTreeClassifier ('--', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best'))
RandomForestClassifier ('.-', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features=1, max_leaf_nodes=None,
                       min_impurity_d

In [45]:
# 데이터 정리
X = df[df.columns[:4]]
X.shape

(150, 4)

In [46]:
# 특정 종에 대해서만 정답을 구축
y = df['species'] == 'versicolor'
type(y), y[:2]

(pandas.core.series.Series, 0    False
 1    False
 Name: species, dtype: bool)

In [65]:
# 훈련용, 테스트용 데이터 분류
# 데이터가 작아서 결과물의 변별력을 높이고자 임의로 80%로 테스트 데이터를 높였다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

In [66]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((30, 4), (120, 4), (30,), (120,))

LogisticRegression : 0.7464698331193839
DecisionTreeClassifier : 0.9492939666238767
RandomForestClassifier : 0.9817073170731708
GaussianNB : 0.9842747111681643


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [80]:
class_map = {
    'LogisticRegression': ('-', LogisticRegression()),
    'DecisionTreeClassifier': ('--', DecisionTreeClassifier(max_depth=5)),
    'RandomForestClassifier': ('.-', RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)),
    'GaussianNB': (':', GaussianNB()),
}

# 시각화를 통해서 성능 확인(ROC, AUC)
plt.figure(figsize=(8, 8))

# 반복하여 시각화 처리
for name, (lineStyle, model) in class_map.items():
    
    # 학습
    model.fit(X_train, y_train)
    
    # 예측
    # predict_proba() : 예측에 대한 확률
    # 0, 1 중에 1이 될 확률이 0.57%이다. -> 1을 더 많이 예측하는 모델
    preds = model.predict_proba(X_test)
    
    # preds -> Series 타입으로 설정 : roc_curve() 함수 형식에 맞추기 위해서 예측답 추출
    pred = pd.Series(preds[:, 1])
    
    # ROC 값 획득
    # roc 리턴값
    # fpr : 거짓 양성비
    # tpr : 참 양성비
    # _ : 임계값, fpr과 tpr을 계산할 때(내부에서) 사용한 결정함수의 임계값 : 이해
    fpr, tpr, _ = roc_curve(y_test, pred)
    
    # ROC 드로잉
    # AUC 표시 -> 수치
    auc_score = auc(fpr, tpr)
    print('%s : %s' % (name, auc_score))
    
    # 선 그리기
    plt.plot(fpr, tpr, lineStyle, linewidth=4, label=name)
    
    pass

# 시각화 마무리
plt.legend()    # 범례 표시
plt.plot([0, 1], [0, 1], 'k--')    # 대각선
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()    # 표시